In [3]:
import tensorflow as tf
import PIL
import os
import numpy as np
import pandas as pd
import extract_faces_keep_in_mem
from search_videos_in_directory import search_videos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
input_direc = input("Enter the absolute path of the input directory: ")
output_direc = input("Enter the absolute path of the output directory: ")
try:
    os.mkdir(output_direc)
except FileExistsError:
    pass
search_videos(input_direc, output_direc)

Enter the absolute path of the input directory: /home/debanik/downloaded_videos
Enter the absolute path of the output directory: /home/debanik/PycharmProjects/Detecting-Deepfakes/temp


TypeError: search_videos() takes from 0 to 1 positional arguments but 2 were given

In [2]:
#input_direc = '/home/debanik/downloaded_videos'
#output_direc = '/home/debanik/PycharmProjects/Detecting-Deepfakes/temp'

In [3]:
# create generator
datagen = ImageDataGenerator(featurewise_center=True, 
                             featurewise_std_normalization=True, 
                             rotation_range=10, 
                             horizontal_flip = True, 
                             validation_split=0.2)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(output_direc, class_mode='binary', batch_size=64)
# confirm the iterator works
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 9 images belonging to 2 classes.
Batch shape=(9, 256, 256, 3), min=0.000, max=255.000
